In [1]:
import wrds
from database.adatabase import ADatabase
import pandas as pd
from tqdm import tqdm
from time import sleep

In [2]:
db = wrds.Connection(wrds_username='ejchung')
market = ADatabase("market")

Loading library list...
Done


In [3]:
price_table = "stkdlysecuritydata"
stock_data = "stocknames"

In [4]:
stock_names = db.get_table("crsp","stocknames",columns=["permno","ticker"]).groupby(["permno","ticker"]).mean().reset_index()

In [10]:
market.connect()
market.store("stock_names",stock_names)
market.disconnect()

In [6]:
db.describe_table("crsp","stkdlysecuritydata")

Approximately 105286752 rows in crsp.stkdlysecuritydata.


,name,nullable,type,comment
0,permno,True,INTEGER,PERMNO
1,yyyymmdd,True,INTEGER,YYYYMMDD - Daily Calendar Period Key
2,dlycaldt,True,DATE,Daily Calendar Date
3,dlydelflg,True,VARCHAR(1),Daily Delisting Flag
4,dlyprc,True,"NUMERIC(13, 6)",Daily Price
5,dlyprcflg,True,VARCHAR(2),Daily Price Flag
6,dlycap,True,"NUMERIC(13, 2)",Daily Capitalization
7,dlycapflg,True,VARCHAR(2),Daily Capitalization Flag
8,dlyprevprc,True,"NUMERIC(13, 6)",Daily Previous Price
9,dlyprevprcflg,True,VARCHAR(2),Daily Previous Price Flag


In [7]:
columns=['permno', 'yyyymmdd',"dlyprc","dlyprcvol"]
market.connect()
market.drop("historical_prices")
for permno in tqdm(stock_names["permno"].unique()):
    try:
        price = db.raw_sql(f"""select permno, dlycaldt, dlyprc, dlyprcvol 
                            from crsp.stkdlysecuritydata 
                            where permno = {permno} and
                            dlycaldt>='01/01/2000'""", 
                         date_cols=['dlycaldt'])
        price = price.merge(stock_names,on="permno").drop("permno",axis=1).rename(columns={"dlycaldt":"date","dlyprc":"adjclose","dlyprcvol":"volume"}).fillna(0)
        price["adjclose"] = [float(str(x)) for x in price["adjclose"]]
        price["volume"] = [float(str(x)) for x in price["volume"]]
        price["date"] = pd.to_datetime(price["date"],format="%Y%m%d")
        market.store("historical_prices",price)
    except:
        continue
market.create_index("historical_prices","ticker")
market.disconnect()

  0%|                                                                                                                                                                                                                                               | 0/36764 [00:00<?, ?it/s]

market historical_prices documents must be a non-empty list


  0%|                                                                                                                                                                                                                                       | 8/36764 [00:00<35:10, 17.42it/s]

market historical_prices documents must be a non-empty list
market historical_prices documents must be a non-empty list
market historical_prices documents must be a non-empty list
market historical_prices documents must be a non-empty list
market historical_prices documents must be a non-empty list
market historical_prices documents must be a non-empty list
market historical_prices documents must be a non-empty list
market historical_prices documents must be a non-empty list


  0%|                                                                                                                                                                                                                                      | 17/36764 [00:00<23:13, 26.37it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 36639/36764 [35:39<00:11, 11.06it/s]

market historical_prices documents must be a non-empty list


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 36656/36764 [35:40<00:06, 15.84it/s]

market historical_prices documents must be a non-empty list


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36764/36764 [35:49<00:00, 17.10it/s]


In [8]:
price

,date,adjclose,volume,ticker
0,2010-06-29,23.89,4.560125e+08,TSLA
1,2010-06-30,23.83,4.195821e+08,TSLA
2,2010-07-01,21.96,1.840138e+08,TSLA
3,2010-07-02,19.20,1.009104e+08,TSLA
4,2010-07-06,16.11,1.129509e+08,TSLA
...,...,...,...,...
3395,2023-12-22,252.54,2.352373e+10,TSLA
3396,2023-12-26,256.61,2.224827e+10,TSLA
3397,2023-12-27,261.44,2.777820e+10,TSLA
3398,2023-12-28,253.18,2.867281e+10,TSLA


In [9]:
db.close()